In [1]:
import sys
import os
import pandas as pd
import json

#  path to src
sys.path.append(os.path.abspath("../../src"))

from noise import add_combined_noise
from train_bert import train_and_evaluate

metrics_list = []
noise_tag = "noise_25"

for run in range(3):
    print(f"Running {noise_tag} – repetition {run + 1}")

    #  cleaned dataset
    df = pd.read_csv("../../data/cleaning/2_cleaned_data.csv")

    #  relevant fields
    df["text"] = df[["sender", "receiver", "urls", "clean_text"]].fillna("").astype(str).agg(" ".join, axis=1)

    #  BERT input
    df = df[["text", "label"]].rename(columns={"text": "body"})

    # Apply 25% combined noise
    df["body"] = df["body"].apply(lambda x: add_combined_noise(x, noise_level=0.25, seed=42 + run))

    # Run 
    tag = f"{noise_tag}_r{run + 1}"
    train_and_evaluate(df, model_tag=tag, text_col="body")

    # Save metrics
    with open(f"results/metrics_{tag}.json") as f:
        metrics = json.load(f)
        metrics["noise_level"] = noise_tag
        metrics["run"] = run + 1
        metrics_list.append(metrics)

# Save 
df_metrics = pd.DataFrame(metrics_list)
df_metrics.to_csv(f"results/metrics_{noise_tag}.csv", index=False)
df_metrics


Running noise_25 – repetition 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'train_runtime': 64.7913, 'train_samples_per_second': 12.347, 'train_steps_per_second': 1.543, 'train_loss': 0.4457662582397461, 'epoch': 1.0}
Running noise_25 – repetition 2


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'train_runtime': 55.5105, 'train_samples_per_second': 14.412, 'train_steps_per_second': 1.801, 'train_loss': 0.39388973236083985, 'epoch': 1.0}
Running noise_25 – repetition 3


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'train_runtime': 49.8733, 'train_samples_per_second': 16.041, 'train_steps_per_second': 2.005, 'train_loss': 0.5723843002319335, 'epoch': 1.0}


,accuracy,f1_phishing,precision,recall,noise_level,run
0,0.920,0.9298,0.8833,0.9815,noise_25,1
1,0.960,0.9633,0.9545,0.9722,noise_25,2
2,0.905,0.9140,0.8938,0.9352,noise_25,3
